## Reading table using big query python client

In [25]:
from google.cloud import bigquery,storage
import pandas as pd

In [9]:
bg = bigquery.Client(location="US", project="tiger-mle")

#### Fetching data

In [10]:
query = """
    SELECT *
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    limit 5000
"""
query_job = bg.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()
df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,1635cc78e071c55379ba87d3d275a4f21333d213,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 11:30:00+00:00,2016-09-22 11:45:00+00:00,600,0.8,NaN,NaN,NaN,NaN,...,0.0,9.25,Credit Card,Chicago Independents,NaN,NaN,None,NaN,NaN,None
1,1e3168900f8090e1bab555b15210f6ad701af327,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 11:45:00+00:00,2016-09-22 12:00:00+00:00,960,7.6,NaN,NaN,NaN,NaN,...,0.0,21.50,Cash,Chicago Independents,NaN,NaN,None,NaN,NaN,None
2,c620405438b7bfcc6378f885f6aa56d1b8be9369,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 15:00:00+00:00,2016-09-22 15:45:00+00:00,3000,13.2,NaN,NaN,NaN,NaN,...,0.0,37.00,Cash,Chicago Independents,NaN,NaN,None,NaN,NaN,None
3,c8b56b9c599580efc1e4ffe36ac36cfb7ea85cbd,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 17:15:00+00:00,2016-09-22 17:45:00+00:00,2220,12.2,NaN,NaN,NaN,NaN,...,4.5,46.58,Credit Card,Chicago Independents,NaN,NaN,None,NaN,NaN,None
4,c876619a0a2869039533144d7105948e9cfc1951,1158f25979ad78fd3dafc867a540ad761b65922c312e61...,2016-09-19 21:00:00+00:00,2016-09-19 21:30:00+00:00,1560,15.6,NaN,NaN,NaN,NaN,...,4.0,52.40,Credit Card,Sun Taxi,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,c0d1ea2f19cf0d59fc1111be6eb06aa48ba7c11b,d72104d59ef45d3ecb85925a234e9366c59ab32d9751d1...,2016-09-27 21:15:00+00:00,2016-09-27 21:15:00+00:00,300,0.9,NaN,NaN,NaN,NaN,...,1.0,10.25,Credit Card,City Service,NaN,NaN,None,NaN,NaN,None
4996,09825857f726e2082bc34b8a575b126607b6fb62,d72104d59ef45d3ecb85925a234e9366c59ab32d9751d1...,2016-09-27 19:15:00+00:00,2016-09-27 19:30:00+00:00,660,0.8,NaN,NaN,NaN,NaN,...,0.0,9.75,Credit Card,City Service,NaN,NaN,None,NaN,NaN,None
4997,514cd988da83929607dade5d1f19b73c086f1c13,d72104d59ef45d3ecb85925a234e9366c59ab32d9751d1...,2016-09-27 19:30:00+00:00,2016-09-27 19:45:00+00:00,360,0.8,NaN,NaN,NaN,NaN,...,1.0,9.25,Credit Card,City Service,NaN,NaN,None,NaN,NaN,None
4998,a04461b4cb1fcaf3dcd0dd24f175e3afd8c78f15,d72104d59ef45d3ecb85925a234e9366c59ab32d9751d1...,2016-09-27 22:15:00+00:00,2016-09-27 22:30:00+00:00,480,1.4,NaN,NaN,NaN,NaN,...,1.5,9.00,Cash,City Service,NaN,NaN,None,NaN,NaN,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   unique_key              5000 non-null   object             
 1   taxi_id                 5000 non-null   object             
 2   trip_start_timestamp    5000 non-null   datetime64[ns, UTC]
 3   trip_end_timestamp      5000 non-null   datetime64[ns, UTC]
 4   trip_seconds            5000 non-null   int64              
 5   trip_miles              5000 non-null   float64            
 6   pickup_census_tract     2 non-null      float64            
 7   dropoff_census_tract    4 non-null      float64            
 8   pickup_community_area   0 non-null      float64            
 9   dropoff_community_area  0 non-null      float64            
 10  fare                    5000 non-null   float64            
 11  tips                    5000 non-null   flo

In [22]:
df = df[['unique_key', 'taxi_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds', 'trip_miles', 'fare', 'tips', 'tolls', 'extras', 'trip_total', 'payment_type', 'company']]

In [23]:
df.describe()

,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total
count,5000.000000,5000.000000,5000.000000,5000.000000,4617.000000,5000.000000,5000.000000
mean,889.938200,5.311712,15.048028,1.520508,0.000074,1.040630,17.803834
std,2244.448627,11.751022,100.602549,2.671032,0.004105,3.271346,101.490111
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,360.000000,1.000000,6.050000,0.000000,0.000000,0.000000,7.250000
50%,600.000000,1.900000,8.500000,0.000000,0.000000,0.000000,10.250000
75%,1020.000000,5.500000,14.500000,2.000000,0.000000,1.000000,16.750000
max,86346.000000,223.200000,7067.150000,20.000000,0.270000,72.000000,7098.750000


In [24]:
df_company = df[['company','trip_seconds','trip_miles','fare','tips','tolls','extras','trip_total']].groupby(
            ['company'],as_index = False).sum()
df_company.head()

,company,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total
0,24 Seven Taxi,246300,1158.30,3950.75,506.66,0.0,375.0,4881.41
1,303 Taxi,191368,985.40,2446.75,170.44,0.0,42.0,2659.19
2,Checker Taxi Affiliation,13440,58.00,248.00,35.90,0.0,67.5,353.90
3,Chicago Carriage Cab Corp,778875,8959.86,9914.00,993.06,0.0,810.0,11874.56
4,Chicago Independents,131220,637.40,2049.25,181.10,0.0,146.0,2397.35


#### Save dataframe to Parquet format

In [5]:
df.to_parquet('df_parquet.gzip',compression='gzip')

#### Reading data from Parquet file

In [6]:
df1 = pd.read_parquet('df_parquet.gzip')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   unique_key              5000 non-null   object             
 1   taxi_id                 5000 non-null   object             
 2   trip_start_timestamp    5000 non-null   datetime64[ns, UTC]
 3   trip_end_timestamp      5000 non-null   datetime64[ns, UTC]
 4   trip_seconds            5000 non-null   int64              
 5   trip_miles              5000 non-null   float64            
 6   pickup_census_tract     2 non-null      float64            
 7   dropoff_census_tract    4 non-null      float64            
 8   pickup_community_area   0 non-null      float64            
 9   dropoff_community_area  0 non-null      float64            
 10  fare                    5000 non-null   float64            
 11  tips                    5000 non-null   flo

#### Storing data in a bucket

In [7]:
import os
print(os.getcwd())

/home/jupyter/MLE-GCP


In [14]:
client = storage.Client()
bucket = client.get_bucket('mle-gcp-tbk-1')

In [13]:
bucket.blob('chicago_taxi_trips/data/test.gzip').upload_from_string(df1.to_parquet(), 'text/gzip')

#### Reading data from a bucket

In [15]:
import pyarrow.parquet as pq
import gcsfs

In [19]:
fs = gcsfs.GCSFileSystem(project='tiger-mle')

f = fs.open('gs://mle-gcp-tbk-1/chicago_taxi_trips/data/test.gzip')
#df_parquet = pq.ParquetFile(f)
df2 = pd.read_parquet(f)


In [20]:
df2.head()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,1635cc78e071c55379ba87d3d275a4f21333d213,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 11:30:00+00:00,2016-09-22 11:45:00+00:00,600,0.8,NaN,NaN,NaN,NaN,...,0.0,9.25,Credit Card,Chicago Independents,NaN,NaN,None,NaN,NaN,None
1,1e3168900f8090e1bab555b15210f6ad701af327,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 11:45:00+00:00,2016-09-22 12:00:00+00:00,960,7.6,NaN,NaN,NaN,NaN,...,0.0,21.50,Cash,Chicago Independents,NaN,NaN,None,NaN,NaN,None
2,c620405438b7bfcc6378f885f6aa56d1b8be9369,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 15:00:00+00:00,2016-09-22 15:45:00+00:00,3000,13.2,NaN,NaN,NaN,NaN,...,0.0,37.00,Cash,Chicago Independents,NaN,NaN,None,NaN,NaN,None
3,c8b56b9c599580efc1e4ffe36ac36cfb7ea85cbd,d2602cb4eab81d9227cd17a415cec4099309d84ff8520c...,2016-09-22 17:15:00+00:00,2016-09-22 17:45:00+00:00,2220,12.2,NaN,NaN,NaN,NaN,...,4.5,46.58,Credit Card,Chicago Independents,NaN,NaN,None,NaN,NaN,None
4,c876619a0a2869039533144d7105948e9cfc1951,1158f25979ad78fd3dafc867a540ad761b65922c312e61...,2016-09-19 21:00:00+00:00,2016-09-19 21:30:00+00:00,1560,15.6,NaN,NaN,NaN,NaN,...,4.0,52.40,Credit Card,Sun Taxi,NaN,NaN,None,NaN,NaN,None
